In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def detect_features(img1, img2, algorithm='SIFT'):

    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Feature Detection and Matching
    sift = cv2.SIFT_create()
    
    if algorithm=='SIFT':
        keypoints1, descriptors1 = sift.detectAndCompute(gray1, None)
        keypoints2, descriptors2 = sift.detectAndCompute(gray2, None)
        
    elif algorithm=='ORB':
        orb = cv2.ORB_create()
        keypoints1 = orb.detect(gray1, None)
        keypoints2 = orb.detect(gray2, None)

        keypoints1, descriptors1 = sift.compute(gray1, keypoints1, None)
        keypoints2, descriptors2 = sift.compute(gray2, keypoints2, None)
    
    else:
        print("detector not implemented!")
        exit()

    return keypoints1, keypoints2, descriptors1, descriptors2


In [12]:
def match_features(descriptors1, descriptors2, threshold=0.5):

    # Feature Matching
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)

    # define the minimum count of matches
    # MIN_MATCH_COUNT = 10
    
    # Apply Lowe's ratio test to select good matches
    good_matches = []
    for m, n in matches:
        if m.distance < threshold * n.distance:
            good_matches.append(m)

    match_rate = len(good_matches) / len(matches)

    # Print count of good matches
    # print(len(good_matches))

    return good_matches, match_rate

In [4]:
def recover_homography(keypoints1, keypoints2, good_matches):

    # Robust Recovery of Homography using RANSAC
    dst_left = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_right = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    H, mask = cv2.findHomography(dst_left, dst_right, cv2.RANSAC, 5.0)
    matches_mask = mask.ravel().tolist()

    return H, mask, matches_mask

In [5]:
def image_stitching(img_left, img_right, H):

    # Image Stitching
    # Normalize images to avoid noise.
    img_left = cv2.normalize(img_left.astype('float'), None, 
                            0.0, 1.0, cv2.NORM_MINMAX)   
    img_right = cv2.normalize(img_right.astype('float'), None, 
                            0.0, 1.0, cv2.NORM_MINMAX)   
    
    # left image
    height_l, width_l, _ = img_left.shape
    corners = [[0, 0, 1], [width_l, 0, 1], [width_l, height_l, 1], [0, height_l, 1]]
    corners_new = [np.dot(H, corner) for corner in corners]
    corners_new = np.array(corners_new).T 
    x_news = corners_new[0] / corners_new[2]
    y_news = corners_new[1] / corners_new[2]
    y_min = min(y_news)
    x_min = min(x_news)

    translation_mat = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]])
    H = np.dot(translation_mat, H)
    
    # Get height, width
    height_new = int(round(abs(y_min) + height_l))
    width_new = int(round(abs(x_min) + width_l))
    dsize = (width_new, height_new)

    warped_l = cv2.warpPerspective(img_left, H, dsize)


    # right image

    height_r, width_r, _ = img_right.shape
    
    height_new = int(round(abs(y_min) + height_r))
    width_new = int(round(abs(x_min) + width_r))
    dsize = (width_new, height_new)
    

    warped_r = cv2.warpPerspective(img_right, translation_mat, dsize)
     
    pixel_black = [0, 0, 0]  # Black pixel.
    
    # Stitching procedure, store results in warped_l.
    for i in range(warped_r.shape[0]):
        for j in range(warped_r.shape[1]):
            pixel_l = warped_l[i, j, :]
            pixel_r = warped_r[i, j, :]
            
            if not np.array_equal(pixel_l, pixel_black) and np.array_equal(pixel_r, pixel_black):
                warped_l[i, j, :] = pixel_l
            elif np.array_equal(pixel_l, pixel_black) and not np.array_equal(pixel_r, pixel_black):
                warped_l[i, j, :] = pixel_r
            elif not np.array_equal(pixel_l, pixel_black) and not np.array_equal(pixel_r, pixel_black):
                warped_l[i, j, :] = (pixel_l + pixel_r) / 2
            else:
                pass
                  
    stitch_image = warped_l[:warped_r.shape[0], :warped_r.shape[1], :]
    return stitch_image

In [ ]:
def experiment(img_left, img_right, threshold, algorithm):
    if not img_left.shape == img_right.shape:
        img_right = cv2.resize(img_right, (img_left.shape[1], img_left.shape[0]))

    keypoints1, keypoints2, descriptors1, descriptors2 = detect_features(img_left, img_right, mask=False, algorithm=algorithm)
    good_matches, match_rate = match_features(descriptors1, descriptors2, threshold=threshold)
    H, mask, matches_mask = recover_homography(keypoints1, keypoints2, good_matches)

    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                       singlePointColor = None,
                       matchesMask = matches_mask, # draw only inliers
                       flags = 2)
    
    mask_img = cv2.drawMatches(img_left, keypoints1, img_right, keypoints2, good_matches, None, **draw_params)

    result = image_stitching(img_left, img_right, H)

    plt.figure(figsize=(20, 10), dpi=300)
    plt.subplot(1, 2, 1)
    plt.title('(a) panorama image')
    plt.imshow(result)
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(mask_img, cv2.COLOR_BGR2RGB))
    plt.title('(b) matched image (match rate:{:.4f})'.format(match_rate))
    plt.show()

In [11]:
# load 
base_path = 'image_pairs/'
img_left_list = ['image_pairs_01_01.jpg', 'image_pairs_02_02.png', 'image_pairs_03_01.jpg', 'image_pairs_04_01.jpg']
img_right_list = ['image_pairs_01_02.jpg', 'image_pairs_02_01.png', 'image_pairs_03_02.jpg', 'image_pairs_04_02.jpg']
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

for i in range(0, len(img_left_list)):
    img_left = cv2.imread(base_path + img_left_list[i])
    img_right = cv2.imread(base_path + img_right_list[i])

    for threshold in thresholds:

        try:
        
        


ValueError: too many values to unpack (expected 2)